<a href="https://colab.research.google.com/github/rroshanii/KenLMPersianSpellingEditor/blob/main/KenLMPersianSpellingEditor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!rm -rf /path/to/unwanted/files/*

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-datasets tensorflow datasets

In [7]:
import os
from google.colab import auth
auth.authenticate_user()
os.environ['HF_TOKEN'] = 'G#?#x"Kh-m+]8g!'

In [8]:
from datasets import load_dataset

# بارگیری فقط 1 درصد از داده‌های اخبار فارسی
#https://saied71.github.io/RohanAiLab/2021/09/03/Demo-datasets.html

persian_news = load_dataset("saied/persian_news_dataset", split="train[:1%]")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


persian_news_dataset.py:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

The repository for saied/persian_news_dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/saied/persian_news_dataset.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


persian_news_dataset.zip:   0%|          | 0.00/4.52G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5113718 [00:00<?, ? examples/s]

In [9]:
print(persian_news)

Dataset({
    features: ['text', 'title', 'category'],
    num_rows: 51137
})


In [10]:
from datasets import load_dataset

# تعریف تابع پیش‌پردازش
def preprocess_text(text):
    cleaned_text = text.replace('\n', ' ').strip()  # حذف خط‌های جدید و فضای اضافی
    return cleaned_text

# پیش‌پردازش تمام متن‌ها
cleaned_news = [preprocess_text(item['text']) for item in persian_news]  # تغییر به `persian_news` بدون کلید 'train'


In [19]:
# ذخیره‌سازی داده‌های پیش‌پردازش‌شده در یک فایل
with open("cleaned_persian_news.txt", "w", encoding="utf-8") as f:
    for news in cleaned_news:
        f.write(news + "\n")


In [ ]:
##########################################
#          نصب  KenLM Model
###########################################

In [18]:
!git clone https://github.com/kpu/kenlm.git
!cd /content/kenlm
!mkdir build
!cd /content/kenlm/
!cmake /content/kenlm/
!make -j 4


[  1%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum-dtoa.cc.o
[  2%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum.cc.o
[  3%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fast-dtoa.cc.o
[  5%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/cached-powers.cc.o
[  6%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fixed-dtoa.cc.o
[  7%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/strtod.cc.o
[  8%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/double-to-string.cc.o
[ 10%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/string-to-double.cc.o
[ 11%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/chain.cc.o
[ 12%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/count_records.cc.o
[ 13%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/io.cc.o
[ 15%] Building CXX obje

In [20]:
# آموزش مدل با KenLM
!/content/bin/lmplz -o 5 < cleaned_persian_news.txt > model.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/cleaned_persian_news.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 18335336 types 346009
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:4152108 2:1061794368 3:1990864512 4:3185383168 5:4645350400
Statistics:
1 346009 D1=0.741348 D2=1.00228 D3+=1.25778
2 3914048 D1=0.780984 D2=1.09193 D3+=1.33585
3 10073556 D1=0.864891 D2=1.17665 D3+=1.36798
4 14280357 D1=0.929581 D2=1.30018 D3+=1.43083
5 16098964 D1=0.913977 D2=1.49033 D3+=1.5829
Memory estimate for binary LM:
type      MB
probing  931 assuming -p 1.5
probing 1095 assuming -r models -p 1.5
trie     461 without quantization
trie     258 assuming -q 8 -b 8 quantization 
trie     405 assuming -a 22 array pointer compression
trie     202 assuming -a 22 -q 8 -b 8 array pointer compression and

In [22]:
!pip install https://github.com/kpu/kenlm/archive/master.zip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.6/553.6 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184442 sha256=ebc88891ee1e1f731239f11f1cc94cb6b892d5060c35c94b9dec0f2c7a2fc7c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-an42w8y3/wheels/a5/73/ee/670fbd0cee8f6f0b21d10987cb042291e662e26e1a07026462
Successfully built kenlm


In [1]:
!/content/bin/build_binary model.arpa model.binary


Reading model.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [3]:
!ls /content/

bin			  CMakeCache.txt       drive		  lib	    model.arpa	  util
build			  CMakeFiles	       kenlm		  lm	    model.binary
cleaned_persian_news.txt  cmake_install.cmake  kenlmConfig.cmake  Makefile  sample_data


In [13]:
import kenlm

kenlm.Model('model.binary')

# بارگذاری مدل باینری KenLM
model = kenlm.Model("/content/model.binary")

# امتیازدهی به یک جمله
sentence = "این یک جمله آزمایشی است."
score = model.score(sentence)
print(f"Score: {score}")
# لیستی از نامزدهای جایگزین برای یک جمله
candidates = ["رفت", "رفتم", "می روم"]

best_score = float('-inf')
best_candidate = None

for candidate in candidates:
    sentence = f"من به خانه   {candidate}"
    score = model.score(sentence)
    if score > best_score:
        best_score = score
        best_candidate = candidate

print(f"بهترین کلمه: {best_candidate} با امتیاز: {best_score}")




Score: -24.17621612548828
بهترین کلمه: رفتم با امتیاز: -19.26911163330078
